In [ ]:
# default_exp ppcc

# pico_pi_controller.ppcc
> Command line tool

In [ ]:
# hide
# from nbdev.showdoc import *

In [ ]:
#export
#!/usr/bin/python3

In [ ]:
# export
from sys import byteorder
from platform import node
import threading, _thread
import socket
import argparse, logging
import atexit
import datetime, time
from CircuitPython_pico_pi_common.codes import *

logger = logging.getLogger()
logging.basicConfig(level = logging.DEBUG)

In [ ]:
# export
tcp_address = ('127.0.0.1', 16164)

REG_NAME = {}
REG_INT = []
for name, code in REG_CODE.items():
    REG_NAME[int.from_bytes(code, byteorder=byteorder)] = name
    REG_INT.append(int.from_bytes(code, byteorder=byteorder))
    
def log_txn(fname, message, msg=None):
        """Wrapper for logger."""
        id_str = ID_CODE.decode()
        i2c_str = '|'
        logger.info('%-4s %-7s %-47s %s' % (id_str, fname, message+str(msg or ''), i2c_str))

@atexit.register
def goodbye():
    """Cancel any socket connections"""
    fname='goodbye'
    global ppcc_socket
    if 'ppcc_socket' in locals():
        try:
            ppcc_socket.close()
            log_txn(fname,"TCP socket cleanup completed.")
        except:
            log_txn(fname,"TCP socket error on cleanup.")
    print(fname,"Exiting.")

def connect_ppdd():
    ppcc_socket = socket.socket()
    ppcc_socket.connect(tcp_address)
    print("Connected to localhost")

    command = bytearray([CMD_CODE['FLICKER']])+bytearray([0x13])+bytearray([0x03])+bytearray([0x0A])

    ppcc_socket.sendall(command)
    print("Sent: ",command)

    # Receive the data
    while(True):
        data = ppcc_socket.recv(56)
        if(data==b''):
            print("Connection closed")
            break
        print("Received: ",data.decode())
    ppcc_socket.close()
    
def main():
    fname='main'
    global ppcc_socket
    
    # handle command-line arguments
    parser = argparse.ArgumentParser(description='ppdd argument parser')
    parser.add_argument('command', type=str, help="Command to send")
    parser.add_argument('-c', default='localhost', type=str, help="Default: PPController at localhost")
    parser.add_argument('-a', default='0x13', type=str, help="Default: address is bosmang.")
    parser.add_argument('-b', action="store_true", help="Send command to bosmang")
    
    args, unknown = parser.parse_known_args()
    
    for arg in unknown:
        log_txn(fname,"unrecognized command line arguments: ",arg)
    
    if args.b:
        pass
        
    if args.command == 'flicker':
        print("FLICKER command given")
        
    # setup socket client for messages to & from ppdd
    
    #thread_list  = []
    #thread_count = 1

    #for index in range(thread_count):
    connection_thread = threading.Thread(target=connect_ppdd())
    #    thread_list.append(connection_thread)
        
    #for index in range(thread_count):
    #    thread_list[index].join()

if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt as e:
        # handled by atexit
        #exit(str('Exiting.'))
        pass

INFO:root:ppdd main    unrecognized command line arguments: -f         |


Connected to localhost
Sent:  bytearray(b'z\x13\x03\n')
Received:  Command receivd. Awaiting confirmation from PPController
Connection closed


In [ ]:
%tb

No traceback available to show.


In [ ]:
# hide
try:
    from IPython.display import display, Javascript
    display(Javascript('IPython.notebook.save_checkpoint();'))
    from time import sleep
    sleep(0.3)
    from nbdev.export import notebook2script
    notebook2script()
except ModuleNotFoundError:
    pass
    """CircuitPython kernel has no nbdev"""

<IPython.core.display.Javascript object>

Converted 00_core.ipynb.
Converted 50_ppdd.ipynb.
Converted 60_ppcc.ipynb.
Converted index.ipynb.
